In [ ]:
%pip install -qqq transformers
%pip install fugashi ipadic

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import json
import numpy as np
import pandas as pd
import torch
from transformers import BertJapaneseTokenizer, BertModel


In [2]:
# SentenceBERTを文章からベクトルへの写像に使うためのクラス
class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens-v2", device=device)

def encode(data):
    id, _id, *sentences, label = json.loads(data).values()
    sentence_vectors = model.encode(sentences)
    distance = np.linalg.norm(sentence_vectors[0] - sentence_vectors[1])
    return distance, label

In [ ]:
with open("valid_v1.0.json") as f:
    dataset = f.readlines()

In [ ]:
# datasetの各行のjsonをエンコードしてベクトル間の距離を算出します
distances_labels = pd.DataFrame(map(encode, dataset), columns=["distance", "label"])
distances_labels.distance = distances_labels.distance / np.max(distances_labels.distance) * 5

In [ ]:
corr = distances_labels.corr().loc["distance", "label"]
print(f"相関係数：{corr}")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.scatter(distances_labels.distance, distances_labels.label, color="#6EE7B7")
ax.set(xlabel="predicted distances", ylabel="ground truth")
ax.grid()
ax.legend()
fig.savefig("scatter_diagram.png")